In [1]:
import os
from PIL import Image
import logging
from IPython.display import HTML, display
import math

from itertools import islice, product

In [2]:
# DWORD for segment count
# Each segment lookup has a DWORD for file offset

# Each segment has a DWORD for segment length
# then 5 WORDS:
# x origin
# y origin
# unknown
# width
# height
# followed by the segment length - 9 (bytes):
# look to be pixels runs
# pixel runs start with x offest any y offeset two's complement
# followed by a pixel count word
# then a set of 15-bit hi color RGB words

In [3]:
def log_progress(sequence, every=None, size=None, name='Items'):
    """wrapper around interator to show jupyter loading bar widget for loop"""
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [4]:
def convert_pixel(p):
    """Convert two bytes into tuple for (r, g, b, a)"""
    MAX_VALUE = 255
    value = int(p)
    a = (value & int('1000000000000000', 2)) >> 15
    r = (value & int('0111110000000000', 2)) >> (10)
    g = (value & int('0000001111100000', 2)) >> 5
    b = (value & int('0000000000011111', 2)) >> 0
    return int(MAX_VALUE * r / pow(2, 5)),  int(MAX_VALUE * g / pow(2, 5)), int(MAX_VALUE * b / pow(2, 5)), MAX_VALUE * (1 - a)

In [5]:
def chunk(it, size):
    it = iter(it)
    return list(iter(lambda: tuple(islice(it, size)), ()))

In [6]:
import itertools
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

def tripwise(iterable):
    a, b, c = itertools.tee(iterable, 3)
    next(b, None)
    next(c, None)
    next(c, None)
    return zip(a, b, c)

In [7]:
def format_word(v, w):
    text = '{:02X} {:02X}'.format(v, w)
    if w >= 0x80:
        return f'<b>{text}</b>'
    else:
        r, g, b, a = convert_pixel((w << 8) + v)
        color = f'#{r:02X}{g:02X}{b:02X}'
        return f'<span style="background-color:{color};">{text}</span>'

In [8]:
def hex_view(data):
    text = []
    text.append(
        14 * '&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8)]) +
        '&nbsp;&nbsp;' +
        '&nbsp;'.join(['{:02X}'.format(y) for y in range(8, 16)])
    
    )
    text.append('')
    for i, x in enumerate(chunk(data, 16)):
        text.append('{:08X}:&nbsp;&nbsp;&nbsp;&nbsp;'.format(i * 16) +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[:8:2], x[1:8:2])]) +
              '&nbsp;&nbsp;' +
              '&nbsp;'.join([format_word(y, z) for y, z in zip(x[8:16:2], x[9:16:2])])
             )
    return '<span style="font-family: Courier New, Courier, Lucida Sans Typewriter, Lucida Typewriter, monospace;">{}</span>'.format("<br>".join(text))

def display_hex_view(data):
    display(HTML(hex_view(data)))
    
display_hex_view(bytes(range(132)))

In [9]:
def set_pixel(image, i0, row, p):
    try:
        image[i0, row] = p
    except:
        pass
    
def twos(val):
    """Convert byte to a two complement value"""
    if val & 0b10000000:
        return val - (1<<8)
    return val

    
# some form of RLE
def draw_image(file, index, data, offset, width, height):
    img = Image.new(
            'RGBA', (width, height), (0, 0, 0, 0))
    image = img.load()
    x0, y0 = offset

    image_stream = iter(int.from_bytes(p1, 'little') for p1 in chunk(data, 2))
    while (True) : 
        next_val = next(image_stream, 'end') 
        # if there are no more values in iterator, break the loop
        if next_val == 'end': 
            break
        else:
            second_header = next(image_stream)
            count = next(image_stream)
            row = twos(second_header & 0xff) + y0
            i0 = twos(next_val & 0xff) + x0
            for i in range(count):
                pixel = next(image_stream)
                set_pixel(image, i0, row, convert_pixel(pixel))
                i0 += 1
                if i0 > width:
                    row += 1
                    i0 = 0
    if not os.path.exists(f'{OUTPUT}/{file}'):
        os.makedirs(f'{OUTPUT}/{file}')
    # draw a magenta point at the index offset
    # image[x0, y0] = (255, 0, 255)
    img.save(f'{OUTPUT}{file}/{file}-{index}.png')
#     SCALE = 4
#     if SCALE != 1:
#         img = img.resize((SCALE * width, SCALE * height))
    #display(img)
#     frames[0].save('test.gif', format='GIF',
#                append_images=frames[1:], save_all=True, duration=10, loop=0)

In [10]:
from collections import namedtuple

sprite_header = namedtuple('sprite_header', ['x_origin', 'y_origin', 'unknown', 'width', 'height'])
bounds = namedtuple('bounds', ['left', 'top', 'right', 'bottom'])

In [11]:
# Look at the top of the files
FOLDER = '/home/alex/.wine/drive_c/GOG Games/WH40K Final Liberation/CEL'
OUTPUT = './sprites/'

def convert_files():
    for filepath in log_progress(sorted(os.listdir(FOLDER))):
        filesize = os.path.getsize(FOLDER + '/' + filepath)
        filename = filepath.split('.')[0]
        if int(filename[4:]):
            with open(FOLDER + '/' + filepath, 'rb') as file:
                header = file.read(29)  # 30 byte header
                sep = file.read(1)
                assert sep == b'\x1a', sep  # 26 in every file
                internal_filesize = file.read(4)
                assert filesize == int.from_bytes(internal_filesize, 'little') 
                segments = int.from_bytes(file.read(4), 'little')
                print(f'file {filename[4:]}')
                if filesize > segments * 4 + 38:
                    offsets = []
                    for x in range(segments):
                        offsets.append(int.from_bytes(file.read(4), 'little'))
                    headers, chunks = [], []
                    for index, (s, e) in enumerate(pairwise(offsets)):
                        seg_length = int.from_bytes(file.read(4), 'little')
                        if seg_length > 0:
                            seg_header = [
                                x for x in chunk(file.read(10), 2)
                            ]
                            c = file.read(seg_length-14)
                            assert seg_length == e - s, "mismatch {} {}".format(seg_length, e - s)
                            h = sprite_header(
                                int.from_bytes(seg_header[0], 'little', signed=True), 
                                int.from_bytes(seg_header[1], 'little', signed=True), 
                                int.from_bytes(seg_header[2], 'little'), 
                                int.from_bytes(seg_header[3], 'little'), 
                                int.from_bytes(seg_header[4], 'little')
                            )
                            headers.append(h)
                            chunks.append(c)
                            print(f'index {index}', h)
                        # display_hex_view(c[:256])
                        else:
                            pass
                            #print("Segment of zero length!")

                    # bounds to
                    #       _
                    #       |
                    # |-----x----|
                    #       |
                    #       -
                    max_bounds = bounds(0, 0, 0, 0)
                    for h in headers:
                        if h.x_origin - 1 > max_bounds.left:
                            max_bounds = max_bounds._replace(left=h.x_origin - 1)
                        if h.y_origin - 1 > max_bounds.top:
                            max_bounds = max_bounds._replace(top=h.y_origin - 1)
                        if h.width - h.x_origin - 1 > max_bounds.right:
                            max_bounds = max_bounds._replace(right=h.width - h.x_origin - 1)
                        if h.height - h.y_origin - 1 > max_bounds.bottom:
                            max_bounds = max_bounds._replace(bottom=h.height - h.y_origin - 1)

                    final_width, final_height = max_bounds.left + max_bounds.right + 1, max_bounds.top + max_bounds.bottom + 1
                    print(max_bounds)
                    for index, (h, c) in enumerate(zip(headers, chunks)):
                        try:
                            draw_image(
                                filename[4:], index, 
                                c, 
                                (max_bounds.left,max_bounds.top), 
                                final_width, final_height)
                        except Exception as e:
                            print("failed to draw", str(e))
                else:
                    print("File is Empty")
            print()

In [12]:
try:
    convert_files()
except:
    import pdb
    pdb.post_mortem()

file 0001
index 0 sprite_header(x_origin=20, y_origin=58, unknown=0, width=46, height=58)
index 1 sprite_header(x_origin=22, y_origin=57, unknown=0, width=50, height=58)
index 2 sprite_header(x_origin=33, y_origin=61, unknown=0, width=76, height=62)
index 3 sprite_header(x_origin=38, y_origin=73, unknown=0, width=77, height=74)
index 4 sprite_header(x_origin=26, y_origin=51, unknown=0, width=51, height=52)
index 5 sprite_header(x_origin=22, y_origin=56, unknown=0, width=51, height=57)
index 6 sprite_header(x_origin=34, y_origin=69, unknown=0, width=76, height=70)
index 7 sprite_header(x_origin=24, y_origin=62, unknown=0, width=51, height=63)
index 8 sprite_header(x_origin=16, y_origin=94, unknown=0, width=32, height=95)
index 9 sprite_header(x_origin=9, y_origin=63, unknown=0, width=17, height=64)
index 10 sprite_header(x_origin=24, y_origin=57, unknown=0, width=43, height=60)
index 11 sprite_header(x_origin=36, y_origin=74, unknown=0, width=60, height=78)
index 12 sprite_header(x_orig

index 8 sprite_header(x_origin=0, y_origin=120, unknown=6, width=32, height=135)
index 9 sprite_header(x_origin=0, y_origin=100, unknown=22, width=32, height=131)
index 10 sprite_header(x_origin=0, y_origin=63, unknown=32, width=12, height=96)
index 11 sprite_header(x_origin=32, y_origin=132, unknown=0, width=32, height=138)
index 12 sprite_header(x_origin=0, y_origin=123, unknown=8, width=32, height=132)
index 13 sprite_header(x_origin=0, y_origin=123, unknown=65529, width=32, height=117)
index 14 sprite_header(x_origin=0, y_origin=109, unknown=17, width=32, height=126)
index 15 sprite_header(x_origin=0, y_origin=83, unknown=15, width=32, height=99)
index 16 sprite_header(x_origin=0, y_origin=124, unknown=8, width=32, height=133)
index 17 sprite_header(x_origin=0, y_origin=89, unknown=15, width=20, height=105)
bounds(left=31, top=194, right=31, bottom=32)

file 0099
index 0 sprite_header(x_origin=0, y_origin=7, unknown=0, width=77, height=13)
index 1 sprite_header(x_origin=0, y_origin

failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 

file 0120
index 0 sprite_header(x_origin=0, y_origin=78, unknown=0, width=32, height=79)
index 1 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 2 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 3 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 4 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 5 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 6 sprite_head

failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 

file 0199
index 0 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 1 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 2 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 3 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 4 sprite_header(x_origin=0, y_origin=46, unknown=0, width=32, height=47)
index 5 sprite_header(x_origin=0, y_origin=34, unknown=0, width=32, height=35)
index 6 sprite_header(x_origin=0, y_origin=34, unknown=0, width=8, height=35)
index 7 sprite_header(x_origin=-24, y_origin=46, unknown=0, width=8, 

failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 
failed to draw 

file 0201
index 0 sprite_header(x_origin=0, y_origin=78, unknown=0, width=32, height=79)
index 1 sprite_header(x_origin=0, y_origin=78, unknown=0, width=32, height=79)

...


file 3569
index 0 sprite_header(x_origin=5, y_origin=2, unknown=52, width=13, height=48)
index 1 sprite_header(x_origin=5, y_origin=2, unknown=52, width=13, height=75)
index 2 sprite_header(x_origin=24, y_origin=1, unknown=44, width=39, height=45)
index 3 sprite_header(x_origin=39, y_origin=1, unknown=39, width=57, height=41)
index 4 sprite_header(x_origin=59, y_origin=1, unknown=39, width=77, height=52)
index 5 sprite_header(x_origin=48, y_origin=1, unknown=32, width=69, height=34)
index 6 sprite_header(x_origin=53, y_origin=1, unknown=22, width=74, height=26)
index 7 sprite_header(x_origin=78, y_origin=1, unknown=22, width=99, height=26)
index 8 sprite_header(x_origin=48, y_origin=1, unknown=25, width=69, height=30)
index 9 sprite_header(x_origin=40, y_origin=2, unknown=30, width=58, height=33)
index 10 sprite_header(x_origin=63, y_origin=16, unknown=30, width=81, height=47)
index 11 sprite_header(x_origin=25, y_origin=7, unknown=26, width=39, height=39)
index 12 sprite_header(x_ori


file 3571
index 0 sprite_header(x_origin=14, y_origin=9, unknown=45, width=22, height=52)
index 1 sprite_header(x_origin=14, y_origin=8, unknown=45, width=22, height=59)
index 2 sprite_header(x_origin=27, y_origin=8, unknown=42, width=42, height=49)
index 3 sprite_header(x_origin=38, y_origin=7, unknown=38, width=61, height=43)
index 4 sprite_header(x_origin=44, y_origin=7, unknown=38, width=67, height=43)
index 5 sprite_header(x_origin=48, y_origin=6, unknown=31, width=74, height=35)
index 6 sprite_header(x_origin=53, y_origin=2, unknown=20, width=79, height=31)
index 7 sprite_header(x_origin=64, y_origin=2, unknown=20, width=90, height=31)
index 8 sprite_header(x_origin=47, y_origin=3, unknown=26, width=74, height=35)
index 9 sprite_header(x_origin=37, y_origin=9, unknown=26, width=61, height=44)
index 10 sprite_header(x_origin=47, y_origin=17, unknown=26, width=71, height=52)
index 11 sprite_header(x_origin=25, y_origin=13, unknown=25, width=43, height=51)
index 12 sprite_header(x_


file 3574
index 0 sprite_header(x_origin=6, y_origin=1, unknown=52, width=14, height=50)
index 1 sprite_header(x_origin=10, y_origin=1, unknown=52, width=20, height=79)
index 2 sprite_header(x_origin=30, y_origin=1, unknown=44, width=43, height=48)
index 3 sprite_header(x_origin=48, y_origin=1, unknown=39, width=63, height=42)
index 4 sprite_header(x_origin=67, y_origin=1, unknown=39, width=82, height=54)
index 5 sprite_header(x_origin=58, y_origin=1, unknown=32, width=76, height=34)
index 6 sprite_header(x_origin=62, y_origin=1, unknown=22, width=80, height=24)
index 7 sprite_header(x_origin=79, y_origin=1, unknown=22, width=97, height=29)
index 8 sprite_header(x_origin=56, y_origin=1, unknown=25, width=76, height=27)
index 9 sprite_header(x_origin=47, y_origin=9, unknown=26, width=63, height=36)
index 10 sprite_header(x_origin=65, y_origin=22, unknown=26, width=81, height=49)
index 11 sprite_header(x_origin=30, y_origin=14, unknown=26, width=42, height=42)
index 12 sprite_header(x_o


file 3577
index 0 sprite_header(x_origin=9, y_origin=31, unknown=21, width=20, height=53)
index 1 sprite_header(x_origin=10, y_origin=31, unknown=21, width=21, height=53)
index 2 sprite_header(x_origin=28, y_origin=36, unknown=13, width=47, height=50)
index 3 sprite_header(x_origin=45, y_origin=34, unknown=8, width=71, height=43)
index 4 sprite_header(x_origin=45, y_origin=34, unknown=8, width=71, height=43)
index 5 sprite_header(x_origin=58, y_origin=30, unknown=3, width=87, height=33)
index 6 sprite_header(x_origin=64, y_origin=26, unknown=3, width=94, height=29)
index 7 sprite_header(x_origin=63, y_origin=26, unknown=3, width=93, height=29)
index 8 sprite_header(x_origin=57, y_origin=32, unknown=3, width=87, height=35)
index 9 sprite_header(x_origin=45, y_origin=39, unknown=3, width=71, height=43)
index 10 sprite_header(x_origin=45, y_origin=39, unknown=3, width=71, height=43)
index 11 sprite_header(x_origin=28, y_origin=45, unknown=3, width=47, height=49)
index 12 sprite_header(x_


file 4002
index 0 sprite_header(x_origin=0, y_origin=127, unknown=0, width=96, height=128)
index 1 sprite_header(x_origin=0, y_origin=120, unknown=0, width=64, height=121)
index 2 sprite_header(x_origin=0, y_origin=110, unknown=0, width=64, height=111)
index 3 sprite_header(x_origin=0, y_origin=141, unknown=0, width=96, height=142)
index 4 sprite_header(x_origin=0, y_origin=85, unknown=0, width=57, height=86)
index 5 sprite_header(x_origin=0, y_origin=91, unknown=0, width=54, height=92)
index 6 sprite_header(x_origin=0, y_origin=84, unknown=0, width=54, height=85)
bounds(left=0, top=140, right=95, bottom=0)

file 4003
index 0 sprite_header(x_origin=21, y_origin=34, unknown=65533, width=39, height=32)
index 1 sprite_header(x_origin=21, y_origin=36, unknown=65531, width=39, height=33)
index 2 sprite_header(x_origin=15, y_origin=35, unknown=65529, width=30, height=29)
index 3 sprite_header(x_origin=22, y_origin=26, unknown=65529, width=41, height=21)
index 4 sprite_header(x_origin=15, y_

# Files

- 0002, old building tiles
- 0040, silo and pipes
- 0099, old gui elements
- 0292, pipes
- 0293, desert tiles and rock formations
- 0294, pipes and tanks
- 0295, bridge parts
- 0296, tank factory walls
- 0297, more walls
- 0298, eagles
- 0299, ruins
- 0300, chemical tank buildings
- 0301, barrels and crates
- 0302, digger
- 0303, large crashed ship
- 0304, oil well
- 0305, unused warehouse building
- 0306, spaceport
- 0307, rail bridge
- 0308, spaceport
- 0309, spaceport
- 0310, monastery
- 0311, unused building floor tiles
- 0312, unused ruins
- 0313, rubble
- 0314, monastery
- 0315, low walls
- 0316, cooling tower
- 0317, spaceport??
- 0318, regular building brown roof
- 0319, windowed ruins
- 0320, windowed ruins
- 0321, oil well?
- 0322, mining machine?
- 0323, gargant works
- 0324, gargant works
- 0325, destroyed chemical tank buildings
- 0326, gargant works (destroyed?)
- 0327, dump truck
- 0328, imperial factory or car park
- 0329, water tower
- 0330, highway bridge
- 0331, small bunker
- 0332, large imperial building (gliches)
- 0333, crashed thunderhawk
- 0334, trees
- 0335, craters
- 0336, tank factory machinery
- 0337, ruined pillar?
- 0400, grain silo
- 0401, misc, grain silo, drill, starport fuel pumps, combine harvester, rail bridge
- 0402, domed silos, crashed harvester
- 0403, tower and pipes (like 0040 but better)
- 0404, prison camp
- 0405, ??? and building from tank factory
- 0406, tank factory, shelves and forklifts
- 0407, ork buildings
- 0408, ork buildings
- 0409, ork buildings
- 0410, space hulk
- 0411, street lamps
- 0412, space hulk crater
- 0413, junk ???
- 0414, wind turbine
- 0415, sand? space hulk crater
- 0416, space hulk crater
- 0420, ork building
- 0421, red cayon rocks
- 0422, solar pannels
- 0430, sand arch
- 0433, misc highway and city parts
- 0501, imperial chapel
- 0502, ruin?
- 0503, imperial building
- 0504, imperial building
- 0505, imperial chapel ruined
- 0506, imperial chapel ruined
- 0507, imperial capital building
- 0508, imperial capital building
- 0509, industrial building
- 0511, railway
- 0512, imperial building
- 0513, imperial building
- 0514, building bits
- 0515, oil rig
- 0516, oil rig
- 0666, bunker

- 0871, target reticle
- 0872, smoke
- 0873, 4 rockets
- 0874, 2 big rockets
- 0875, 6 rockets
- 0876, blue laser
- 0877, large shell
- 0878, manticore rockets
- 0879, whirlwind rockets
- 0880, smoke
- 0881, infantry heavy weapon rocket
- 0882, small shell
- 0883, yellow flame or laser
- 0884, red laser
- 0885, blue laser
- 0886, infantry heavy weapon rocket 1 pixel
- 0887, random black specks
- 0888, large grey shell
- 0889, round bronze shell
- 0890, round bronze shell with red glow light
- 0891, blue selection
- 0892, green selection
- 0893, red selection
- 0894, windmill
- 0895, green glowy thing
- 0896, empty
- 0897, green bubble
- 0898, green bubble
- 0899, empty
- 0970, empty
- 0975, objective tokens
- 0976, capaign banners
- 0983, empty
- 0984, vortex
- 0985, empty
- 0986, mekboy dragster shield
- 0987, pulsa rocket explode
- 0988, pulsa rocket pulse
- 0989, pulsa rocket
- 0990, old ruins
- 0992, old lasers
- 0993, explosion
- 0994, laser
- 0995, explosion
- 0996, huge explosion
- 0997, terrain steam and oil well fountains
- 0998, flame
- 0999, gargant flames and explosions

- 1000+, imperial war engines
- 1100+, imperial vehicles
- 1200+, infantry
- 1500+, titan
- 2100+, space marines vehicle
- 2200+, space marine infantry
- 3000+, ork war engines
- 3100+, ork vehicles
- 3200+, ork infantry + bikes
- 3502+, ork gargants
- 3550+, ork gargant weapons
- 3565+, imperial weapons

- 4000, oil well
- 4001, spaceport
- 4002, chemical tanks
- 4003, barrels and boxes
- 4005, chapel
- 4006, large space ship crashed
- 4007, digger

## infantry sprites



## vehicle sprites

0. W body
1. W wheel 1
2. W wheel 2
3. W dead
4. W gun 1
5. W gun 2
6. NWW body
7. NWW wheel 1
8. NWW wheel 2
9. SE body
10. SE wheel 1
11. SE wheel 2
12. SE dead
13. SE gun 1
14. SE gun 2
15. NNW body
16. NNW wheel 1
17. NNW wheel 2
18. NW body
19. NW wheel 1
20. NW wheel 2
21. NW dead
22. NW gun 1
23. NW gun 2
24. SEE body
25. SEE wheel 1
26. SEE wheel 2
27. S body
28. S wheel 1
29. S wheel 2
30. S dead
31. S gun 1
32. S gun 2
33. SSE body
34. SSE wheel 1
35. SSE wheel 2
36. N body
37. N wheel 1
38. N wheel 2
39. N dead
40. N gun 1
41. N gun 2
42. W gun 3
43. SE gun 3
44. NW gun 3
45. S gun 3